In [ ]:
# analytics.ipynb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# Connect to database
engine = create_engine('mssql+pyodbc://...')

# Load data
df = pd.read_sql("""
    SELECT w.WasteTypeId, wt.Name, w.EstimatedKg, w.CreatedAt, m.Name as Municipality
    FROM WasteReports w
    JOIN WasteTypes wt ON w.WasteTypeId = wt.WasteTypeId
    JOIN Municipalities m ON w.MunicipalityId = m.MunicipalityId
""", engine)

# Analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Waste composition
waste_composition = df.groupby('Name')['EstimatedKg'].sum()
axes[0,0].pie(waste_composition.values, labels=waste_composition.index, autopct='%1.1f%%')
axes[0,0].set_title('Waste Composition')

# Time series analysis
df['Date'] = pd.to_datetime(df['CreatedAt']).dt.date
daily_volume = df.groupby('Date')['EstimatedKg'].sum()
axes[0,1].plot(daily_volume.index, daily_volume.values)
axes[0,1].set_title('Daily Waste Volume')
axes[0,1].set_xlabel('Date')
axes[0,1].set_ylabel('Weight (kg)')

# Municipality comparison
municipality_volume = df.groupby('Municipality')['EstimatedKg'].sum()
axes[1,0].bar(municipality_volume.index, municipality_volume.values)
axes[1,0].set_title('Waste Volume by Municipality')
axes[1,0].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()